In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('Home_datatest.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
# Get numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns

# Fill missing values for numeric columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Encode categorical variables
df = pd.get_dummies(df, drop_first=True)

# Define X as the current columns after encoding
X = df.copy()

# Ensure the new dataset has the same columns as the training data
df = df.reindex(columns=X.columns, fill_value=0)

from sklearn.preprocessing import StandardScaler

# Fit the scaler on X and transform
scaler = StandardScaler()
df_scaled = scaler.fit_transform(X)

In [14]:
from sklearn.ensemble import GradientBoostingRegressor

# Load and preprocess the training data
train_df = pd.read_csv('Home_data train.csv')

# Fill missing values for numeric columns
train_numeric_cols = train_df.select_dtypes(include=['number']).columns
train_df[train_numeric_cols] = train_df[train_numeric_cols].fillna(train_df[train_numeric_cols].median())

# Encode categorical variables
train_df = pd.get_dummies(train_df, drop_first=True)

# Separate target before aligning columns
y_train = train_df['SalePrice']
X_train = train_df.drop('SalePrice', axis=1)

# Align columns with test data
X_train = X_train.reindex(columns=X.columns, fill_value=0)

# Scale X_train using the same scaler as test data
X_train_scaled = scaler.fit_transform(X_train)

# Train the model on the training data
from xgboost import XGBRegressor
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)
# Make predictions on the test data
rf_predictions = gbr.predict(df_scaled)


In [16]:
# Ensure rf_predictions is defined
if 'rf_predictions' in locals():
    # Add predictions to the DataFrame
    df['SalePrice'] = rf_predictions  
    
    # Save only the 'id' and 'Predicted_SalePrice' columns to a CSV file
    if 'Id' in df.columns:
        df[['Id', 'SalePrice']].to_csv('Predicted_House_Prices.csv', index=False)
        print("Predictions saved to Predicted_House_Prices.csv")
    else:
        print("Error: 'id' column not found in the dataset.")
else:
    print("Error: rf_predictions is not defined. Please run the cell that generates rf_predictions.")

Predictions saved to Predicted_House_Prices.csv
